In [21]:
# импортируем необходимые пакеты и  функции
from mpi4py import MPI
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
from matplotlib import cm
from PIL import Image
import os, sys, glob
%matplotlib inline

In [33]:
# гравитационная постоянная м^3 кг^-1 с^-2
G = 6.67408E-11 
# размер области 2L x 2L x 2L
L = 1E6
# радиус частиц
R = 5E3
# плотность
rho = 2.5E3
# число частиц
N = 100
# Моделируемый отрезок времени
T0 = 1.0E+6
# Число шагов по времени
NTSteps = 1000
# Интервал между визуализациями данных
VizStep = 100
# Шаг по времени
dT = T0/NTSteps

In [23]:
# функция для измерения времени выполнения (без изменений)
def How_Long(func, args):
    start = MPI.Wtime()
    func(*args)
    stop = MPI.Wtime()        
    hrs = (stop - start)//(60*60)
    mns = ((stop - start) - hrs*60*60)//60
    scs = ((stop - start) - hrs*60*60 - mns*60)
    print (" {} hour {} min {} sec\n".format(hrs,mns,scs))
    
    return hrs, mns, scs, stop - start

In [24]:
# Инициализация начальных условий с помощью numpy, без циклов
def init_cond_numpy(NP):
    crds = L - 2*L*np.random.random(size=(NP, 3))
    rds = np.random.randint(low=100, high=R, size=NP, dtype='int64')
    mss = rho*4/3*np.pi*rds**3
    vls = np.zeros(shape=[NP, 3])
    return crds, rds, mss, vls

In [25]:
# Функция визуализации данных (без изменений)
def Viz(Crds, FileName):
    mpl.rcParams['figure.figsize'] = (24.0, 8.0)
    plt.subplots(ncols=3)
    X = np.array(Crds)[:,0]
    Y = np.array(Crds)[:,1]
    Z = np.array(Crds)[:,2]
    plt.subplot(131)
    plt.title('View From X Axis')
    plt.xlabel('Y')
    plt.ylabel('Z')
    plt.scatter(Y, Z, cmap=cm.hot)
    plt.xlim([-L,L])
    plt.ylim([-L,L])
    plt.subplot(132)
    plt.title('View From Y Axis')
    plt.xlabel('X')
    plt.ylabel('Z')
    plt.scatter(X, Z, cmap=cm.hot)
    plt.xlim([-L,L])
    plt.ylim([-L,L])
    plt.subplot(133)
    plt.title('View From Z Axis')
    plt.xlabel('X')
    plt.ylabel('Y')    
    plt.scatter(X, Y, cmap=cm.hot)
    plt.xlim([-L,L])
    plt.ylim([-L,L])
    plt.savefig(FileName,dpi=300, bbox_inches='tight')
    plt.close()

In [26]:
# Основная функция для решения задачи N тел, немного оптимизирована
def nbody_numpy(NP, NT, dt, crds, vls, mss):  
    
    for step in range(1, NT + 1):
        A = np.zeros(shape=[NP, 3])
        for i in range(NP):
            dr = np.delete(crds, i, axis=0) - crds[i]
            dr3 = np.linalg.norm(dr, axis=1)**3
            A[i] = np.dot(np.delete(mss, i, axis=0)/dr3, dr)
        vls += dt*G*A
        crds += dt*vls
        
        # сохраняем только изображения png
        if step % VizStep == 0:
            print ("STEP: {}".format(step))
            fn = os.path.join("WORK_SPACE", "ResNBody_List","%020d.png" % step)
            Viz(crds,fn)

In [27]:
# Та же функция с распараллеливанием
# Для простоты считаем, что число частиц всегда делится нацело на число процессов
def nbody_numpy_parallel(NP, NT, dt, crds, vls, mss):
    from ipyparallel import Client
    client = Client()
    dview = client[:]
    dview.block = True
    dview['part'] = NP//len(dview.targets) # Число частиц, обрабатываемых одним процессом
    dview['G'] = G
    dview['mss'] = mss
    dview.scatter('id', client.ids, flatten=True)
    for step in range(1, NT + 1):
        dview['crds'] = crds
        dview['vls'] = vls
        dview.execute(
            '''
            import numpy as np
            A = np.zeros(shape=[part, 3])
            for i in range(part):
                dr = np.delete(crds, i + id*part, axis=0) - crds[i + id*part]
                dr3 = np.linalg.norm(dr, axis=1)**3
                A[i] = np.dot(np.delete(mss, i + id*part, axis=0)/dr3, dr)
            '''
        )
        vls += dt*G*dview.gather('A')
        crds += dt*vls
        
        # сохраняем только изображения png
        if step % VizStep == 0:
            print ("STEP: {}".format(step))
            fn = os.path.join("WORK_SPACE", "ResNBody_List","%020d.png" % step)
            Viz(crds,fn)

In [34]:
# Генерация начального состояния системы
coords, rads, mass, vels = init_cond_numpy(N)
fn = os.path.join("WORK_SPACE", "ResNBody_List", "%020d.png" % 0)
Viz(coords, fn)

In [35]:
How_Long(nbody_numpy_parallel, [N, NTSteps, dT, coords, vels, mass])
fp_in = "WORK_SPACE/ResNBody_List/*.png"
fp_out = "WORK_SPACE/ResNBody_List.gif"

img, *imgs = [Image.open(f) for f in sorted(glob.glob(fp_in))]
img.save(fp=fp_out, format='GIF', append_images=imgs,
         save_all=True, optimize=False, duration=200, loop=0)

STEP: 100
STEP: 200
STEP: 300
STEP: 400
STEP: 500
STEP: 600
STEP: 700
STEP: 800
STEP: 900
STEP: 1000
 0.0 hour 0.0 min 44.34010800009128 sec

